<a href="https://colab.research.google.com/github/sneha4948/Federated-Learning-for-Supply-Chain-Sensor-Data-Processing/blob/main/3_layer_MLP_Experiment_with_Simulated_data_flower_local.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Installing FLOWER, other requirements

In [8]:
!pip install -q "flwr[simulation]" flwr-datasets

In [2]:
# you might see a warning after running the command below, this can be ignored
# if you are running this outside Colab, you probably need to adjust the command below
# !pip install torch==1.13.1+cpu torchvision==0.14.1+cpu --extra-index-url https://download.pytorch.org/whl/cpu

In [3]:
!pip install matplotlib

### Dataset Simulation

In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
from statsmodels.tsa.api import VAR
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
# Flower Simulation code
import flwr as fl
from flwr.common import Context

In [ ]:
# Generate random data within given ranges
temperature = np.random.uniform(8, 14, n_samples)
humidity = np.random.uniform(80, 95, n_samples)
co2 = np.random.uniform(600, 2000, n_samples)

# Function to label condition
def label_condition(temp, hum, co2):
    if temp > 12 and hum > 90 and co2 > 1500:
        return "Poor Condition"
    elif (temp > 10 and hum > 85) or (co2 > 1200):
        return "Average Condition"
    else:
        return "Good Condition"

# Create labels
labels = [label_condition(temp, hum, co2) for temp, hum, co2 in zip(temperature, humidity, co2)]

# Create a DataFrame
df = pd.DataFrame({
    'Temperature': temperature,
    'Humidity': humidity,
    'CO2': co2,
    'Condition': labels
})

# Save dataset to CSV
df.to_csv('container_conditions.csv', index=False)
print("Dataset created and saved as container_conditions.csv")


Dataset created and saved as container_conditions.csv


### Data Preprocessing for Classification

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Load dataset
data = pd.read_csv('container_conditions.csv')

# Encode the labels to integers
le = LabelEncoder()
data['Condition'] = le.fit_transform(data['Condition'])

# Split data into features and labels
X = data[['Temperature', 'Humidity', 'CO2']]
y = data['Condition']

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
import pandas as pd
df=pd.read_csv("container_conditions.csv")

## Client

In [23]:
# Define Flower client
class ContainerClient(fl.client.NumPyClient):
    def __init__(self, model, X_train, y_train, X_test, y_test):
        self.model = model
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test

    def get_parameters(self, config=None):  # Add config argument here
        return self.model.get_weights()

    def fit(self, parameters, config):
        self.model.set_weights(parameters)
        self.model.fit(self.X_train, self.y_train, epochs=1, batch_size=32, verbose=0)
        return self.model.get_weights(), len(self.X_train), {}

    def evaluate(self, parameters, config):
        self.model.set_weights(parameters)
        loss, accuracy = self.model.evaluate(self.X_test, self.y_test)
        return accuracy, len(self.X_test), {"loss": loss, "accuracy": accuracy}


# Correct client_fn signature
def client_fn(context: Context):
    model = create_model()
    return NumPyClient.to_client(ContainerClient(model, X_train, y_train, X_test, y_test))

### FedAvg, FedProx, FedBuff, FedAsync Strategy

In [24]:
from flwr.server.strategy import FedAvg, FedProx
from flwr.server.server import ServerConfig
from flwr.client import NumPyClient


In [25]:
# Define FedBuff strategy
class FedBuffStrategy(FedAvg):
    def __init__(self, buffer_size=2, **kwargs):
        super().__init__(**kwargs)
        self.buffer_size = buffer_size
        self.buffer = []

    def aggregate_fit(self, rnd, results, failures):
        # Buffer the results
        self.buffer.extend(results)
        if len(self.buffer) >= self.buffer_size:
            # Aggregate when buffer is full
            aggregated_parameters, aggregated_metrics = super().aggregate_fit(rnd, self.buffer, failures)
            self.buffer = []  # Reset buffer
            return aggregated_parameters, aggregated_metrics
        else:
            # Return None to indicate that aggregation is not yet performed
            return None, {}


In [26]:
from flwr.server.strategy import Strategy
from flwr.common import Parameters, FitRes, EvaluateRes, Scalar
from typing import Optional, Tuple, List, Dict
import numpy as np

class FedAsyncStrategy(Strategy):
    def __init__(self, eta: float = 0.1):
        self.current_weights: Optional[List[np.ndarray]] = None
        self.eta = eta  # Learning rate for asynchronous updates

    def initialize_parameters(self, client_manager):
        # Initialize global model parameters
        if self.current_weights is None:
            # Create a new model to get initial weights
            model = create_model()
            self.current_weights = model.get_weights()
        parameters = fl.common.weights_to_parameters(self.current_weights)
        return parameters

    def configure_fit(
        self, rnd: int, parameters: Parameters, client_manager
    ) -> List[Tuple[fl.server.client_proxy.ClientProxy, fl.common.FitIns]]:
        # Configure clients to train asynchronously
        config = {}
        # Sample all available clients
        clients = client_manager.all()
        fit_ins = fl.common.FitIns(parameters, config)
        instructions = [(client, fit_ins) for client in clients]
        return instructions

    def configure_evaluate(
        self, rnd: int, parameters: Parameters, client_manager
    ) -> List[Tuple[fl.server.client_proxy.ClientProxy, fl.common.EvaluateIns]]:
        # Optionally, configure clients for evaluation
        # For this example, we will not perform evaluation
        return []

    def aggregate_fit(
        self, rnd: int, results: List[Tuple[fl.server.client_proxy.ClientProxy, FitRes]], failures
    ) -> Tuple[Optional[Parameters], Dict[str, Scalar]]:
        # Update global weights asynchronously
        if not results:
            return None, {}

        for _, fit_res in results:
            client_weights = fl.common.parameters_to_weights(fit_res.parameters)
            if self.current_weights is None:
                self.current_weights = client_weights
            else:
                # Asynchronous update: weighted average
                self.current_weights = [
                    (1 - self.eta) * w_global + self.eta * w_client
                    for w_global, w_client in zip(self.current_weights, client_weights)
                ]

        parameters = fl.common.weights_to_parameters(self.current_weights)
        return parameters, {}

    def aggregate_evaluate(
        self, rnd: int, results: List[Tuple[fl.server.client_proxy.ClientProxy, EvaluateRes]], failures
    ) -> Tuple[Optional[float], Dict[str, Scalar]]:
        # Optionally, aggregate evaluation results
        # Since we're not performing evaluation, return None
        return None, {}

    def evaluate(self, parameters: Parameters):
        # Optional: Implement server-side evaluation if needed
        return None


In [27]:
def start_federated_learning(strategy_name: str, num_rounds: int):
    """Start federated learning with the given number of rounds."""

    # Strategy for the server, using the FedAvg algorithm
    if strategy_name == 'fedavg':
        strategy = FedAvg(
            fraction_fit=1.0,  # Train on all clients each round
            fraction_evaluate=1.0,  # Evaluate on all clients each round
            min_fit_clients=5,  # Minimum number of clients to participate in training
            min_evaluate_clients=5,  # Minimum number of clients to participate in evaluation
            min_available_clients=5,  # Minimum number of available clients needed
        )

    elif strategy_name == 'fedprox':
        strategy = FedProx(
            fraction_fit=1.0,  # Train on all clients each round
            fraction_evaluate=1.0,  # Evaluate on all clients each round
            min_fit_clients=5,  # Minimum number of clients to participate in training
            min_evaluate_clients=5,  # Minimum number of clients to participate in evaluation
            min_available_clients=5,  # Minimum number of available clients needed
            proximal_mu=0.1,  # FedProx proximal term parameter (mu)
    )
    elif strategy_name == 'fedbuff':
        strategy = FedBuffStrategy(
                buffer_size=2,
                fraction_fit=1.0,
                fraction_evaluate=1.0,
                min_fit_clients=2,  # Adjust based on buffer_size
                min_evaluate_clients=2,
                min_available_clients=5,
            )
    elif strategy_name == 'fedasync':
        strategy = FedAsyncStrategy(eta=0.1)


    # Set the number of rounds in ServerConfig
    server_config = ServerConfig(num_rounds=num_rounds)

    # Run the federated learning simulation for `num_rounds`
    fl.simulation.start_simulation(
        client_fn=client_fn,
        num_clients=5,  # Number of clients
        config=server_config,  # Pass the server configuration for the number of rounds
        client_resources={"num_cpus": 1}  # Adjust based on your system resources
    )


# Run 64

In [28]:
# Creating model function
# 3 layerr 64
def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.InputLayer(shape=(3,)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(64,activation='relu'),
        tf.keras.layers.Dense(3, activation='softmax')  # 3 output classes
    ])
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

## FedAvg

In [29]:
# Run the federated learning simulation for 5 rounds (or any desired number of rounds)
start_federated_learning(strategy_name='fedavg', num_rounds=5)

	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=5, no round_timeout
2024-11-27 20:59:52,113	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'node:__internal_head__': 1.0, 'node:172.28.0.12': 1.0, 'object_store_memory': 3997345382.0, 'memory': 7994690766.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 2 acto

7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6768 - loss: 5.0086  
1/7 ━━━━━━━━━━━━━━━━━━━━ 3s 599ms/step - accuracy: 0.7188 - loss: 3.2480
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6768 - loss: 5.0086  


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 2]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6880 - loss: 1.4196   [repeated 4x across cluster]


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 3]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)


...

7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6768 - loss: 5.3710   [repeated 4x across cluster]
1/7 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.7188 - loss: 2.8654
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.6768 - loss: 5.3710


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 5]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7332 - loss: 1.2408   [repeated 4x across cluster]


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 5 round(s) in 68.03s
INFO :      	History (loss, distributed):
INFO :      		round 1: 0.675000011920929
INFO :      		round 2: 0.6850000023841858
INFO :      		round 3: 0.7049999833106995
INFO :      		round 4: 0.675000011920929
INFO :      		round 5: 0.7200000286102295
INFO :      


1/7 ━━━━━━━━━━━━━━━━━━━━ 3s 553ms/step - accuracy: 0.8125 - loss: 0.8018
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7332 - loss: 1.2408  


## FedProx

In [30]:
# Run the federated learning simulation for 5 rounds (or any desired number of rounds)
start_federated_learning(strategy_name='fedprox', num_rounds=5)

	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=5, no round_timeout
2024-11-27 21:01:23,065	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'memory': 7994213991.0, 'object_store_memory': 3997106995.0, 'node:__internal_head__': 1.0, 'node:172.28.0.12': 1.0, 'CPU': 2.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 2 acto

7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.6972 - loss: 2.5947  


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 2]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)


1/7 ━━━━━━━━━━━━━━━━━━━━ 2s 340ms/step - accuracy: 0.7812 - loss: 0.7475
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6972 - loss: 2.5947  


INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)


...

INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 5]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)


1/7 ━━━━━━━━━━━━━━━━━━━━ 3s 541ms/step - accuracy: 0.7188 - loss: 0.9162
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6768 - loss: 2.0218  


INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6768 - loss: 2.6589   [repeated 4x across cluster]


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 5 round(s) in 60.26s
INFO :      	History (loss, distributed):
INFO :      		round 1: 0.6850000023841858
INFO :      		round 2: 0.675000011920929
INFO :      		round 3: 0.675000011920929
INFO :      		round 4: 0.675000011920929
INFO :      		round 5: 0.675000011920929
INFO :      


## FedBuff

In [31]:
# Run the federated learning simulation for 5 rounds (or any desired number of rounds)
start_federated_learning(strategy_name='fedbuff', num_rounds=5)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=5, no round_timeout
2024-11-27 21:02:43,495	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'memory': 7994066535.0, 'object_store

7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6768 - loss: 9.3273  


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 2]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)


1/7 ━━━━━━━━━━━━━━━━━━━━ 2s 341ms/step - accuracy: 0.7188 - loss: 4.2655
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6768 - loss: 9.3273  


INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)


...

INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 4]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6768 - loss: 2.5480 [repeated 5x across cluster]
1/7 ━━━━━━━━━━━━━━━━━━━━ 5s 834ms/step - accuracy: 0.7188 - loss: 1.3022
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6768 - loss: 2.5480  
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.6768 - loss: 2.5480


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 5]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6768 - loss: 2.5480   [repeated 2x across cluster]
1/7 ━━━━━━━━━━━━━━━━━━━━ 2s 448ms/step - accuracy: 0.7812 - loss: 0.6848 [repeated 2x across cluster]
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7118 - loss: 0.9258  


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 5 round(s) in 50.47s
INFO :      	History (loss, distributed):
INFO :      		round 1: 0.675000011920929
INFO :      		round 2: 0.699999988079071
INFO :      		round 3: 0.675000011920929
INFO :      		round 4: 0.675000011920929
INFO :      		round 5: 0.7049999833106995
INFO :      


## FedAsync

In [32]:
# Run the federated learning simulation for 5 rounds (or any desired number of rounds)
start_federated_learning(strategy_name='fedasync', num_rounds=5)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=5, no round_timeout
2024-11-27 21:03:51,437	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'node:172.28.0.12': 1.0, 

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6768 - loss: 2.4279  


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 2]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6768 - loss: 3.9412   [repeated 5x across cluster]
1/7 ━━━━━━━━━━━━━━━━━━━━ 5s 915ms/step - accuracy: 0.7188 - loss: 2.5423
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.6768 - loss: 3.9412  
1/7 ━━━━━━━━━━━━━━━━━━━━ 4s 770ms/step - accuracy: 0.7188 - loss: 2.5423
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.6768 - loss: 3.9412  


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 3]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)


...

1/7 ━━━━━━━━━━━━━━━━━━━━ 5s 991ms/step - accuracy: 0.7812 - loss: 0.6765
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6969 - loss: 0.9861   [repeated 4x across cluster]
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.6959 - loss: 1.5315  


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 5]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)


1/7 ━━━━━━━━━━━━━━━━━━━━ 5s 937ms/step - accuracy: 0.7812 - loss: 0.6765
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6768 - loss: 3.0341   [repeated 4x across cluster]
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6959 - loss: 1.5315  


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 5 round(s) in 45.35s
INFO :      	History (loss, distributed):
INFO :      		round 1: 0.675000011920929
INFO :      		round 2: 0.675000011920929
INFO :      		round 3: 0.6949999928474426
INFO :      		round 4: 0.6850000023841858
INFO :      		round 5: 0.675000011920929
INFO :      


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6768 - loss: 3.0341  


# Run 28

In [33]:
# Creating model function
def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.InputLayer(shape=(3,)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(128,activation='relu'),
        tf.keras.layers.Dense(3, activation='softmax')  # 3 output classes
    ])
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [34]:
# Correct client_fn signature
def client_fn(context: Context):
    model = create_model()
    return NumPyClient.to_client(ContainerClient(model, X_train, y_train, X_test, y_test))

## FedAvg

In [35]:
# Run the federated learning simulation for 5 rounds (or any desired number of rounds)
start_federated_learning(strategy_name='fedavg', num_rounds=5)

	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=5, no round_timeout
2024-11-27 21:04:52,760	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'node:172.28.0.12': 1.0, 'node:__internal_head__': 1.0, 'object_store_memory': 3996625305.0, 'memory': 7993250612.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 2 acto

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6768 - loss: 11.1622 
1/7 ━━━━━━━━━━━━━━━━━━━━ 2s 446ms/step - accuracy: 0.7188 - loss: 7.3419
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6768 - loss: 11.1622 


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 2]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6768 - loss: 11.1622  [repeated 2x across cluster]
1/7 ━━━━━━━━━━━━━━━━━━━━ 5s 858ms/step - accuracy: 0.7188 - loss: 2.7650 [repeated 2x across cluster]
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6768 - loss: 11.1622 
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6768 - loss: 5.9916  


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 3]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)


...

INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 4]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6768 - loss: 2.3050   [repeated 5x across cluster]
1/7 ━━━━━━━━━━━━━━━━━━━━ 2s 449ms/step - accuracy: 0.7188 - loss: 1.3228
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6768 - loss: 2.3050  


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 5]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7270 - loss: 1.2129   [repeated 4x across cluster]


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 5 round(s) in 47.28s
INFO :      	History (loss, distributed):
INFO :      		round 1: 0.675000011920929
INFO :      		round 2: 0.675000011920929
INFO :      		round 3: 0.675000011920929
INFO :      		round 4: 0.675000011920929
INFO :      		round 5: 0.7300000190734863
INFO :      


## FedProx

In [36]:
# Run the federated learning simulation for 5 rounds (or any desired number of rounds)
start_federated_learning(strategy_name='fedprox', num_rounds=5)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=5, no round_timeout
2024-11-27 21:05:55,829	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'object_store_memory': 3996723609.0, 

1/7 ━━━━━━━━━━━━━━━━━━━━ 2s 439ms/step - accuracy: 0.7188 - loss: 9.4048
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6768 - loss: 12.9696 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6768 - loss: 12.9696 


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 2]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)


1/7 ━━━━━━━━━━━━━━━━━━━━ 2s 424ms/step - accuracy: 0.7188 - loss: 9.4048
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6768 - loss: 12.9696 
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6768 - loss: 1.6955   [repeated 3x across cluster]
1/7 ━━━━━━━━━━━━━━━━━━━━ 4s 724ms/step - accuracy: 0.7188 - loss: 1.0681
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6768 - loss: 1.6955  


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 3]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)


...

INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 4]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6768 - loss: 2.1829   [repeated 5x across cluster]
1/7 ━━━━━━━━━━━━━━━━━━━━ 2s 462ms/step - accuracy: 0.7188 - loss: 1.1063
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6768 - loss: 2.1829  


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 5]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6768 - loss: 2.3500   [repeated 4x across cluster]


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 5 round(s) in 44.76s
INFO :      	History (loss, distributed):
INFO :      		round 1: 0.675000011920929
INFO :      		round 2: 0.675000011920929
INFO :      		round 3: 0.675000011920929
INFO :      		round 4: 0.675000011920929
INFO :      		round 5: 0.675000011920929
INFO :      


## FedBuff

In [37]:
# Run the federated learning simulation for 5 rounds (or any desired number of rounds)
start_federated_learning(strategy_name='fedbuff', num_rounds=5)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=5, no round_timeout
2024-11-27 21:06:56,800	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'object_store_memory': 39

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6768 - loss: 4.1595  


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 2]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.6768 - loss: 7.0164   [repeated 5x across cluster]
1/7 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.7188 - loss: 4.5184
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.6768 - loss: 7.0164
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.6768 - loss: 7.0164


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 3]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)


...

INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 5]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)


1/7 ━━━━━━━━━━━━━━━━━━━━ 2s 403ms/step - accuracy: 0.7188 - loss: 2.0691
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6768 - loss: 3.5553  


INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6768 - loss: 1.7840   [repeated 4x across cluster]


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 5 round(s) in 47.05s
INFO :      	History (loss, distributed):
INFO :      		round 1: 0.675000011920929
INFO :      		round 2: 0.675000011920929
INFO :      		round 3: 0.6499999761581421
INFO :      		round 4: 0.675000011920929
INFO :      		round 5: 0.675000011920929
INFO :      


## FedAsync

In [38]:
# Run the federated learning simulation for 5 rounds (or any desired number of rounds)
start_federated_learning(strategy_name='fedasync', num_rounds=5)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=5, no round_timeout
2024-11-27 21:07:57,192	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'node:__internal_head__':

1/7 ━━━━━━━━━━━━━━━━━━━━ 4s 701ms/step - accuracy: 0.7188 - loss: 2.7774
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.6768 - loss: 5.9736  
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6768 - loss: 5.9736  


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 2]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6768 - loss: 5.9736  


INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)


...

INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 4]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)


1/7 ━━━━━━━━━━━━━━━━━━━━ 2s 423ms/step - accuracy: 0.8438 - loss: 0.4602
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6768 - loss: 2.5950   [repeated 4x across cluster]
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7477 - loss: 0.7090  


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 5]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)


1/7 ━━━━━━━━━━━━━━━━━━━━ 2s 425ms/step - accuracy: 0.8438 - loss: 0.4602
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7008 - loss: 1.4123   [repeated 4x across cluster]
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7477 - loss: 0.7090  
1/7 ━━━━━━━━━━━━━━━━━━━━ 2s 393ms/step - accuracy: 0.7812 - loss: 0.6682
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7008 - loss: 1.4123  


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 5 round(s) in 43.52s
INFO :      	History (loss, distributed):
INFO :      		round 1: 0.675000011920929
INFO :      		round 2: 0.675000011920929
INFO :      		round 3: 0.675000011920929
INFO :      		round 4: 0.7300000190734863
INFO :      		round 5: 0.6899999976158142
INFO :      


# Run 256

In [39]:
# Creating model function
#256 with 3 layers
def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.InputLayer(shape=(3,)),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(256,activation='relu'),
        tf.keras.layers.Dense(3, activation='softmax')  # 3 output classes
    ])
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [40]:
# Correct client_fn signature
def client_fn(context: Context):
    model = create_model()
    return NumPyClient.to_client(ContainerClient(model, X_train, y_train, X_test, y_test))

## FedAvg

In [41]:
# Run the federated learning simulation for 5 rounds (or any desired number of rounds)
start_federated_learning(strategy_name='fedavg', num_rounds=5)

	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=5, no round_timeout
2024-11-27 21:08:57,825	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'memory': 7992719771.0, 'object_store_memory': 3996359884.0, 'node:__internal_head__': 1.0, 'node:172.28.0.12': 1.0, 'CPU': 2.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 2 acto

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6932 - loss: 3.5192  


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 2]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6768 - loss: 4.2294   [repeated 5x across cluster]
1/7 ━━━━━━━━━━━━━━━━━━━━ 2s 416ms/step - accuracy: 0.7188 - loss: 2.0297
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6768 - loss: 4.2294  


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 3]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)


...

INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 4]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7008 - loss: 1.3533   [repeated 2x across cluster]
1/7 ━━━━━━━━━━━━━━━━━━━━ 4s 736ms/step - accuracy: 0.7188 - loss: 1.2354
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6768 - loss: 2.8233  


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 5]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6768 - loss: 1.1096   [repeated 5x across cluster]
1/7 ━━━━━━━━━━━━━━━━━━━━ 4s 755ms/step - accuracy: 0.7188 - loss: 0.5925
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6768 - loss: 1.1096  
1/7 ━━━━━━━━━━━━━━━━━━━━ 4s 769ms/step - accuracy: 0.7188 - loss: 0.5925
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.6768 - loss: 1.1096


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 5 round(s) in 44.02s
INFO :      	History (loss, distributed):
INFO :      		round 1: 0.699999988079071
INFO :      		round 2: 0.675000011920929
INFO :      		round 3: 0.675000011920929
INFO :      		round 4: 0.6899999976158142
INFO :      		round 5: 0.675000011920929
INFO :      


## FedProx

In [42]:
# Run the federated learning simulation for 5 rounds (or any desired number of rounds)
start_federated_learning(strategy_name='fedprox', num_rounds=5)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=5, no round_timeout
2024-11-27 21:09:58,903	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'memory': 7993270272.0, '

1/7 ━━━━━━━━━━━━━━━━━━━━ 2s 416ms/step - accuracy: 0.7188 - loss: 6.4002
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6768 - loss: 11.8313 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6768 - loss: 11.8313 


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 2]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)


1/7 ━━━━━━━━━━━━━━━━━━━━ 4s 671ms/step - accuracy: 0.7188 - loss: 3.7880
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6768 - loss: 11.8313  [repeated 3x across cluster]
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6768 - loss: 6.4272  


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 3]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)


...

INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 4]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7008 - loss: 1.2612   [repeated 5x across cluster]


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 5]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)


1/7 ━━━━━━━━━━━━━━━━━━━━ 2s 381ms/step - accuracy: 0.7188 - loss: 1.1885
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7008 - loss: 1.2612   [repeated 4x across cluster]
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6768 - loss: 1.7115  


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 5 round(s) in 40.21s
INFO :      	History (loss, distributed):
INFO :      		round 1: 0.675000011920929
INFO :      		round 2: 0.675000011920929
INFO :      		round 3: 0.675000011920929
INFO :      		round 4: 0.6899999976158142
INFO :      		round 5: 0.675000011920929
INFO :      


## FedBuff

In [43]:
# Run the federated learning simulation for 5 rounds (or any desired number of rounds)
start_federated_learning(strategy_name='fedbuff', num_rounds=5)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=5, no round_timeout


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6768 - loss: 1.7115  


2024-11-27 21:10:56,757	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'node:__internal_head__': 1.0, 'node:172.28.0.12': 1.0, 'memory': 7992547739.0, 'object_store_memory': 3996273868.0, 'CPU': 2.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 2 actors
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
(pid=15702) 2024-11-27 21:11:00.654138: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=15702) 2024-11-27 21:11:00.690596: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register fact

7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.6768 - loss: 14.1485 
1/7 ━━━━━━━━━━━━━━━━━━━━ 4s 765ms/step - accuracy: 0.7188 - loss: 8.5210
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6768 - loss: 14.1485 


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 2]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)


...

7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7217 - loss: 0.6236   [repeated 4x across cluster]
1/7 ━━━━━━━━━━━━━━━━━━━━ 4s 809ms/step - accuracy: 0.7812 - loss: 0.3945
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7217 - loss: 0.6236  


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 5]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6950 - loss: 0.6788   [repeated 4x across cluster]


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 5 round(s) in 45.82s
INFO :      	History (loss, distributed):
INFO :      		round 1: 0.675000011920929
INFO :      		round 2: 0.6949999928474426
INFO :      		round 3: 0.675000011920929
INFO :      		round 4: 0.7149999737739563
INFO :      		round 5: 0.6899999976158142
INFO :      


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6950 - loss: 0.6788  


## FedAsync

In [44]:
# Run the federated learning simulation for 5 rounds (or any desired number of rounds)
start_federated_learning(strategy_name='fedasync', num_rounds=5)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=5, no round_timeout
2024-11-27 21:11:56,414	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'node:__internal_head__':

7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6768 - loss: 6.5563  
1/7 ━━━━━━━━━━━━━━━━━━━━ 4s 696ms/step - accuracy: 0.7188 - loss: 4.6917
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6768 - loss: 6.5563  


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 2]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6768 - loss: 5.2479   [repeated 3x across cluster]
1/7 ━━━━━━━━━━━━━━━━━━━━ 5s 841ms/step - accuracy: 0.7188 - loss: 4.6917
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6768 - loss: 6.5563  


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 3]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)


...

INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6768 - loss: 1.1308   [repeated 3x across cluster]
1/7 ━━━━━━━━━━━━━━━━━━━━ 4s 748ms/step - accuracy: 0.7188 - loss: 0.3955


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 5]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6768 - loss: 0.7007   [repeated 5x across cluster]


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 5 round(s) in 42.12s
INFO :      	History (loss, distributed):
INFO :      		round 1: 0.675000011920929
INFO :      		round 2: 0.675000011920929
INFO :      		round 3: 0.6899999976158142
INFO :      		round 4: 0.675000011920929
INFO :      		round 5: 0.675000011920929
INFO :      
